In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-18 12:38:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1,06M  6,55MB/s    in 0,2s    

2024-07-18 12:38:49 (6,55 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of text: ", len(text))

length of text:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab size: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size:  65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hii there'))
print(decode(encode('hii there')))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

In [9]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} then target is {target}")

when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(data.size(0) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + 1 + block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ', xb.shape, xb.dtype)
print(xb)
print('targets: ', yb.shape, yb.dtype)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} then target is {target}")


inputs:  torch.Size([4, 8]) torch.int64
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:  torch.Size([4, 8]) torch.int64
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] then target is 43
when input is [24, 43] then target is 58
when input is [24, 43, 58] then target is 5
when input is [24, 43, 58, 5] then target is 57
when input is [24, 43, 58, 5, 57] then target is 1
when input is [24, 43, 58, 5, 57, 1] then target is 46
when input is [24, 43, 58, 5, 57, 1, 46] then target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
when input is [44] then target is 53
when input is [44, 53] then target is 56
when input is [44, 53, 56] then target is 1
when input is [44, 53, 56, 1] then target is 58
when inp

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
gen_seq = m.generate(idx, max_new_tokens=100)
print(decode(gen_seq[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)

In [14]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if steps % 10 == 0:
        print(f'loss: {loss.item()}')

loss: 4.692410945892334
loss: 4.71571159362793
loss: 4.630363464355469
loss: 4.722168922424316
loss: 4.737388610839844
loss: 4.509944915771484
loss: 4.707157135009766
loss: 4.62866735458374
loss: 4.599071502685547
loss: 4.646826267242432
loss: 4.6221184730529785
loss: 4.569244384765625
loss: 4.594120979309082
loss: 4.4772257804870605
loss: 4.722170829772949
loss: 4.495169162750244
loss: 4.606575012207031
loss: 4.496219158172607
loss: 4.5546135902404785
loss: 4.523606777191162
loss: 4.551448345184326
loss: 4.432989120483398
loss: 4.422276020050049
loss: 4.420462131500244
loss: 4.40394401550293
loss: 4.497410297393799
loss: 4.333616256713867
loss: 4.353393077850342
loss: 4.382663249969482
loss: 4.440627098083496
loss: 4.348025321960449
loss: 4.349501132965088
loss: 4.360719203948975
loss: 4.409273624420166
loss: 4.312803268432617
loss: 4.240131855010986
loss: 4.183422088623047
loss: 4.347282886505127
loss: 4.267045497894287
loss: 4.197660446166992
loss: 4.258787155151367
loss: 4.39411640

In [15]:
idx = torch.zeros((1, 1), dtype=torch.long)
gen_seq = m.generate(idx, max_new_tokens=1000)
print(decode(gen_seq[0].tolist()))


llo br. ave aviasurf my, mayo t ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whe, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
By bre ndy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n sar; my w, fredeeyong
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, pannd e Yolde Th likl beamen, tofr,
n s Byo tred ceathe, il ivilde w
O ff y
Fivede? ig aiMy, I ivis muofounce herevern outh f athawendesees yof th withind be wameats tsteer y blitow,
Ye m o ditoshyd me, ch rte u hart ararwsa
Wou fe,
INurathoune
IESSARin,
MIOLened sust tl.
S:
NMy BAnind g.
iudshank
An chin is a arokisupxaseru t w ity merwo al LOLo bebte loolld worinero ya l aknge ond thal ttry b's mo ge ck.

gh, inketilllin trewnutud t ar,
WAnt cithapis Zimponcrdistherdrtes saure ' erpoperrposthelind y ss of hef t

In [16]:
import torch
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

1) Simple version

In [20]:
xbow = torch.randn((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

2) Optimized version

In [22]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B, T, C) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2) 

True

3) Softmax version

In [24]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

4) Self-attention

In [37]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
x.shape

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)    
wei = q @ k.transpose(-2, -1)
print(q.shape, k.transpose(-2, -1).shape, wei.shape)

torch.Size([4, 8, 16]) torch.Size([4, 16, 8]) torch.Size([4, 8, 8])


In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)    
wei = q @ k.transpose(-2, -1) * head_size ** 0.5 # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x)  # (B, T, head_size)
out = wei @ v  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)